In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt

#데이터 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/악성 URL/DATA/train_FE')

# 데이터 전처리(단축 url 사용 및 필요없는 것 제외)

In [ ]:
train_df = train.loc[:,['URL', 'label', 'length', 'subdomain_count', 'special_char_count',
       'number_of_meaning_words', 'tld_malicious', 'path_depth',
       'max_numeric_sequence','https', 'blacklist_word_count',
       'digit_count', 'suspicious_keyword_flag']]

In [ ]:
train_df['tld_malicious'] = train_df['tld_malicious']*1
train_df['tld_malicious'].value_counts()

,count
tld_malicious,
0,6225887
1,769169


#데이터 분할

In [ ]:
X,y = train_df.drop(columns=['URL','label']),train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
X_train

,length,subdomain_count,special_char_count,number_of_meaning_words,tld_malicious,path_depth,max_numeric_sequence,https,blacklist_word_count,digit_count,suspicious_keyword_flag
763391,14,0,1,0,0,0,0,0,0,0,0
6629791,14,1,1,0,1,1,0,0,2,0,0
5939613,9,0,0,0,0,0,0,0,0,0,0
4668276,18,0,0,0,0,0,0,0,0,0,0
1295931,21,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1692743,13,0,0,0,0,0,0,0,0,0,0
6550634,16,0,1,0,0,0,0,0,0,0,0
6423388,11,0,0,0,0,0,0,0,0,0,0
6962611,9,0,1,0,0,0,0,0,0,0,0


#하이퍼 파라미터 튜닝(GridSearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion' : ['gini'], #데이터 분할 기준
    'max_depth' : [20,30], #트리의 최대 깊이 (과적합 방지 위해)
    'min_samples_split' : [50,100,200], #노드 분할 최소 샘플 수
    'min_samples_leaf' : [25,50,100], # 리프 노드의 최소 샘플 수
    'max_features' : ['sqrt','log2'] #각 노드에서 사용할 특성 (노드 분할 특징)
}

grid_search = GridSearchCV(DecisionTreeClassifier(),[param_grid], cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train[:100000],y_train[:100000])

print('최적의 파라미터 조합:', grid_search.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
최적의 파라미터 조합: {'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 25, 'min_samples_split': 50}


#결정트리 모델 불러오기

In [ ]:
Decision_model = grid_search.best_estimator_

#ROC AUC로 평가하기

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = Decision_model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.9179898997368688
